<b>Importing Metrics for Cities in the Dallas Fortworth (DFW) Metroplex by webscraping www.bestplaces.net website using beautiful soup</b>

In [ ]:
#Upon inspecting the data in the website, it was determined that the name of the cities were stored in a 'div' tag. We are running a for loop through the div tag to determine the different Dallas cities that are listed under it and passing the values to a list.

dallas_city_results_list=[]
url ='https://www.bestplaces.net/find/zip.aspx?msa=19100&st=tx'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
for div_tag in soup.find_all("div", class_="row mt-3"):
    for a_tag in div_tag.find_all('a'):
        dallas_city_results_list.append(a_tag.attrs['href'])


#Creating a column named url to save the links scraped above. The links are then modfied by correcting the beginning portion
df_cities_test=pd.DataFrame(dallas_city_results_list, columns=["url"])
df_cities_test["url"]=df_cities_test["url"].str.replace("../zip-code/","https://www.bestplaces.net/zip-code/")
x=df_cities_test.url.to_list()



dallas_cities_metrics=[]
n=0
for url in x:
    page = requests.get(url)
    status= page.status_code
    soup = BeautifulSoup(page.content, 'html.parser')
    
    for title in soup.find_all("p", class_="card-title text-center"):
        dallas_cities_metrics.append(title.text)
        
    for population in soup.find_all("p", class_="text-center py-0 my-0", attrs={'style':'font-size:30px;'}):
        dallas_cities_metrics.append(population.text)
            
    for other_metrics in soup.find_all("p", class_="text-center", attrs={'style':'font-size:33px;'}):
        dallas_cities_metrics.append(other_metrics.text)  
    
    print(dallas_cities_metrics[n] + " has status code {} and data scraping complete!".format(status))
    n=n+7

#creating list within list at everything 7th element within 'dallas_cities_metrics' list in order to create a dataframe
dallas_cities_metrics=[dallas_cities_metrics[7*i:7*i+7] for i in range(0,math.ceil(len(dallas_cities_metrics)/7))]


print("Webscraping for Dallas-Fortworth (DFW) Metroplex cities data complete! ")  

In [ ]:
#creating dataframe from list
df_dallas_cities_metrics=pd.DataFrame(dallas_cities_metrics, columns=["City","Population","Unemployment_Rate","Median_Income","Median_Home_Price","Median_Age","Climate_Comfort_Index"])

#Removing % sign from Unemployment_Rate
df_dallas_cities_metrics["Population"]=df_dallas_cities_metrics["Population"].str.replace(",","")

#Removing % sign from Unemployment_Rate
df_dallas_cities_metrics["Unemployment_Rate"]=df_dallas_cities_metrics["Unemployment_Rate"].str.replace("%","")

#Removing $ sign from Median_Income
df_dallas_cities_metrics["Median_Income"]=df_dallas_cities_metrics["Median_Income"].str.replace("$","")
df_dallas_cities_metrics["Median_Income"]=df_dallas_cities_metrics["Median_Income"].str.replace(",","")

#Removing $ sign from Median_Home_Price
df_dallas_cities_metrics["Median_Home_Price"]=df_dallas_cities_metrics["Median_Home_Price"].str.replace("$","")
df_dallas_cities_metrics["Median_Home_Price"]=df_dallas_cities_metrics["Median_Home_Price"].str.replace(",","")

#Removing '/10' from Climate_Comfort_Index
df_dallas_cities_metrics["Climate_Comfort_Index"]=df_dallas_cities_metrics["Climate_Comfort_Index"].str.replace("/10","")

#Creating a clean column with City Information
df_dallas_cities_metrics["City_"]=df_dallas_cities_metrics["City"]
df_dallas_cities_metrics["City_"]=df_dallas_cities_metrics["City_"].str.split('(').str[0]
df_dallas_cities_metrics["City_"]=df_dallas_cities_metrics["City_"].str.strip()

#Creating a clean column with State Information
df_dallas_cities_metrics["State"]=df_dallas_cities_metrics["City"]
df_dallas_cities_metrics["State"]=df_dallas_cities_metrics["State"].str[-6:]
df_dallas_cities_metrics["State"]=df_dallas_cities_metrics["State"].str.strip()

#Creating a clean column with Zipcode Information
df_dallas_cities_metrics["Zipcode"]=df_dallas_cities_metrics["City"]
df_dallas_cities_metrics["Zipcode"]=df_dallas_cities_metrics["Zipcode"].str.split('(').str[1]
df_dallas_cities_metrics["Zipcode"]=df_dallas_cities_metrics["Zipcode"].str.split(')').str[0]
df_dallas_cities_metrics["Zipcode"]=df_dallas_cities_metrics["Zipcode"].str.replace("zip ","")

#Dropping old City Column
df_dallas_cities_metrics=df_dallas_cities_metrics.drop(["City"], axis='columns')


#Renaming newly created 'City_' column to 'City'
df_dallas_cities_metrics.rename(columns={'City_':'Neighborhood'}, inplace=True)

#Re-arranging columns in newly create Dallas Cities Metrics Dataframe
df_dallas_cities_metrics=df_dallas_cities_metrics[['Neighborhood','Zipcode','State','Population','Unemployment_Rate','Median_Income','Median_Home_Price','Median_Age','Climate_Comfort_Index']]
df_dallas_cities_metrics.head()

In [ ]:
#Determining number of rows in newly created dallas cities metrics dataframe
len(df_dallas_cities_metrics)

In [ ]:
#Determining data types of the newly created dataframe
df_dallas_cities_metrics.dtypes

In [ ]:
#Converting data to appropriate data types to use in analysis later
df_dallas_cities_metrics['Population']=pd.to_numeric(df_dallas_cities_metrics['Population'])
df_dallas_cities_metrics['Unemployment_Rate']=pd.to_numeric(df_dallas_cities_metrics['Unemployment_Rate'])
df_dallas_cities_metrics['Median_Income']=pd.to_numeric(df_dallas_cities_metrics['Median_Income'])
df_dallas_cities_metrics['Median_Home_Price']=pd.to_numeric(df_dallas_cities_metrics['Median_Home_Price'])
df_dallas_cities_metrics['Median_Age']=pd.to_numeric(df_dallas_cities_metrics['Median_Age'])
df_dallas_cities_metrics['Climate_Comfort_Index']=pd.to_numeric(df_dallas_cities_metrics['Climate_Comfort_Index'])

<b>The web scraped  DFW statistics data has information for multiple zipcodes within the same neighborhoods. We are now going to create a dataframe that has one row per neighborhood by summarizing the data at Neighborhood level by taking the weighted averages of all statistics using population of the various zipcodes as the weight </b>

In [ ]:
#There are two zipcodes in which poplation is 0
len(df_dallas_cities_metrics[df_dallas_cities_metrics["Population"]==0])

In [ ]:
#Eliminating data from zipcodes where poplation as that wouldn't be part of our analysis
df_dallas_cities_metrics=df_dallas_cities_metrics[df_dallas_cities_metrics["Population"] !=0]

In [ ]:
#Checking whether zipcodes with 0 population have been removed from our base data
len(df_dallas_cities_metrics[df_dallas_cities_metrics["Population"]==0])

In [ ]:
df_dallas_cities_metrics.head()

<b> CREATING BASE DATASET FOR THIS PROJECT </b>

In [ ]:
g = df_dallas_cities_metrics.groupby(['Neighborhood'])
df_dallas_stats = g.apply(lambda x: pd.Series([np.sum(x['Population']),
                                               np.average(x['Unemployment_Rate'], weights=x['Population']), 
                             np.average(x['Median_Income'],weights=x['Population']), 
                             np.average(x['Median_Home_Price'],weights=x['Population']),
                             np.average(x['Median_Age'],weights=x['Population']),
                             np.average(x['Climate_Comfort_Index'],weights=x['Population'])],
                                    index=['Population','Unemployment_Rate','Median_Income','Median_Home_Price','Median_Age','Climate_Comfort_Index']))
df_dallas_stats=df_dallas_stats.reset_index()

df_dallas_stats.head()

In [ ]:
# The following code will assign one zipcode per neighborhood to the base data
df_dallas_cities_metrics_Zipcode=df_dallas_cities_metrics.groupby(['Neighborhood'])['Zipcode'].apply(','.join).reset_index()
df_dallas_cities_metrics_Zipcode['Zipcode_']=df_dallas_cities_metrics_Zipcode['Zipcode'].str.split(',').str[0]
df_dallas_cities_metrics_Zipcode=df_dallas_cities_metrics_Zipcode.drop(['Zipcode'],axis='columns')
df_dallas_cities_metrics_Zipcode.rename(columns={'Zipcode_':'Zipcode'}, inplace=True)
df_dallas_cities_metrics_Zipcode.head()

In [ ]:
df_dallas_stats=pd.merge(df_dallas_stats, df_dallas_cities_metrics_Zipcode, how='left')

In [ ]:
print("#Dataframe with Statistics for Dallas has been created! Dataframename: df_dallas_stats")